### 7. juli

Begynte med å jobbe videre på vektendrignene i 06_07_23.ipynb for å få bin-breddene til å minke mot 0 og øke igjen etter 0. Kom dessverre ikke noe videre så gikk tilbake til tidligere kode og skrev mer utfyllende om de ulike vektendringsplottene og hva de betyr/viser.

Skrev også videre på en påbegynt kode fra tidligere for deep double descent-kurve men for CNN og ikke MLP-nettverks som jeg prøvde på i går. Koden vises nedenfor. Denne tar lang tid å kjøre så derfor lar jeg den kjøre mens jeg endrer mer på de tidligere kodene og skriver forklaringer. Sjekket double descent koden før jeg skulle dra for dagen og den var enda ikke ferdig med å kjøre så fikk ikke sett hvilket output den ga.

I koden 23-24_06_23.ipynb fikk jeg til å plotte vektendringene for vektene med høyest verdi over tid som jeg prøvde på tidligere men ikke fikk til. Det viste seg at vektendringen til de største vektene var mye større enn den gjennomsnittlige vektendringen over alle vektene i hvert lag.

I 01-04_07_23.ipynb endret jeg lagstørrelsene fra 28^2, 128, 100, 10 til 28^2, 28^2, 28^2, 10 for å se om det var noen endring i vektfordelingen med endring i lagstørrelsen. Det viste seg at fordelingene så like ut selv om det jo var litt endring i antallet vektendringer, men ikke drastiske endringer som var særlig merkbare.

Hadde oppdateringsmøte med Anders. Vi diskuterte ulike fordelingsfunksjoner som muligens kan passe til vektfordelingen mellom to epoker i hvert av lagene som jeg fant i 01-04_07_23.ipynb.

For å lettere finne en passende fordelingsfunksjon samlet jeg vektfordelingsfigurene for epoke 0-1, 9-10 og 29-30 i én figur. Dette vises i 08_07_23.ipynb.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange

In [2]:
class CNN(nn.Module):
    def __init__(self, width):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, width, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(width, width, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(width, width, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(width, width, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(width *28 * 28, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
# Define the training function
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# Define the evaluation function
def evaluate(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.0 * correct / len(test_loader.dataset)
    return test_loss, test_accuracy


In [ ]:
# Set the random seed for reproducibility
torch.manual_seed(123)

# Define the data transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Define the width parameters to test
widths = list(range(1, 101))

# Define the sample sizes
sample_sizes = [12500, 25000, 50000]

# Define the label noise percentage
label_noise = 0.2

# Define the number of training epochs
epochs = 20

# Define the batch size
batch_size = 64

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize lists to store the results
test_errors = []

In [ ]:
# Iterate over the sample sizes
for sample_size in sample_sizes:
    # Sample a subset from the training dataset
    subset_indices = torch.randperm(len(train_dataset))[:sample_size]
    train_subset = torch.utils.data.Subset(train_dataset, subset_indices)

    # Add label noise to the subset
    num_noisy_labels = int(label_noise * sample_size)
    noisy_label_indices = torch.randperm(sample_size)[:num_noisy_labels]
    for idx in noisy_label_indices:
        train_subset.dataset.targets[idx] = torch.randint(0, 10, (1,))

    # Create data loaders
    train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize lists to store the test errors for each width
    test_errors_sample = []

    # Iterate over the widths
    for width in widths:
        # Create the model
        model = CNN(width).to(device)

        # Define the criterion and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

        # Train the model
        for epoch in trange(epochs):
            train(model, train_loader, optimizer, criterion, device)

        # Evaluate the model
        test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
        test_errors_sample.append(test_loss)

    # Store the test errors for the current sample size
    test_errors.append(test_errors_sample)

# Convert the test errors to numpy arrays for plotting
test_errors = np.array(test_errors)

In [ ]:
# Plot the results
plt.figure(figsize=(10, 7))
plt.title('Deep Double Descent - MNIST')
plt.xlabel('CNN Width Parameter')
plt.ylabel('Test Error')

# Plot the graphs for each sample size
for i, sample_size in enumerate(sample_sizes):
    plt.plot(widths, test_errors[i], label=f'{sample_size} samples')

plt.legend()
plt.show()